In [1]:
import os
import pandas as pd
import numpy as np

### Load data ###

In [2]:
cpd = {}

for dataset in ["CPU", "Memory", "Disk"]:

    results_dir = "./%s_BD" % dataset
    if dataset not in cpd.keys():
        cpd[dataset] = {} 

    for f in os.listdir("%s" % results_dir):
        # Exclude hidden files (most likely thumbnails)
        if f[:2] != "._":
            print("Loading: %s/%s" % (results_dir, f))
            df = pd.read_csv("%s/%s" % (results_dir, f))
            print(len(df))

            if "aggr" in f:
                # Extract CPD threshold from the filename 
                lthreshold = float(f.split("-")[1])

                df["lthreshold"] = lthreshold
                if "aggr" not in cpd[dataset].keys():
                    cpd[dataset]["aggr"] = pd.DataFrame()
                cpd[dataset]["aggr"] = pd.concat([cpd[dataset]["aggr"], df], sort=False)

            if "indiv" in f:
                if "indiv" not in cpd[dataset].keys():
                    cpd[dataset]["indiv"] = pd.DataFrame()
                cpd[dataset]["indiv"] = pd.concat([cpd[dataset]["indiv"], df], sort=False)

            if "meta" in f:
                cpd[dataset]["meta"] = df

            if "segment-durations" in f:
                cpd[dataset]["segment-durations"] = df

Loading: ./CPU_BD/indiv-25.00
956
Loading: ./CPU_BD/indiv-10.00
3433
Loading: ./CPU_BD/meta
11
Loading: ./CPU_BD/indiv-19.00
1386
Loading: ./CPU_BD/segment-durations
14759
Loading: ./CPU_BD/indiv-13.00
2557
Loading: ./CPU_BD/indiv-16.00
1746
Loading: ./CPU_BD/indiv-22.00
1141
Loading: ./Memory_BD/indiv-25.00
2938
Loading: ./Memory_BD/indiv-10.00
9870
Loading: ./Memory_BD/meta
11
Loading: ./Memory_BD/indiv-19.00
4210
Loading: ./Memory_BD/segment-durations
39093
Loading: ./Memory_BD/indiv-13.00
7145
Loading: ./Memory_BD/indiv-16.00
5477
Loading: ./Memory_BD/indiv-22.00
3225
Loading: ./Disk_BD/indiv-25.00
860
Loading: ./Disk_BD/indiv-10.00
2496
Loading: ./Disk_BD/meta
11
Loading: ./Disk_BD/indiv-19.00
1122
Loading: ./Disk_BD/segment-durations
9413
Loading: ./Disk_BD/indiv-13.00
1710
Loading: ./Disk_BD/indiv-16.00
1349
Loading: ./Disk_BD/indiv-22.00
964


In [5]:
count_df = pd.DataFrame(columns=["CPU: CP #", "Mem: CP #", "Disk: CP #"])
for lthreshold in cpd["CPU"]["indiv"]["min_size"].sort_values().unique():
    cpu_cpd_c = len(cpd["CPU"]["indiv"][(cpd["CPU"]["indiv"]["min_size"] == lthreshold)])
    
    mem_cpd_c = len(cpd["Memory"]["indiv"][(cpd["Memory"]["indiv"]["min_size"] == lthreshold)])
    
    disk_cpd_c = len(cpd["Disk"]["indiv"][(cpd["Disk"]["indiv"]["min_size"] == lthreshold)])
    
    count_df.loc[lthreshold] = (cpu_cpd_c, mem_cpd_c, disk_cpd_c)
    
count_df["Total #"] = count_df.sum(1)
count_df.loc['Total',:] = count_df.sum(0)
display(count_df)

,CPU: CP #,Mem: CP #,Disk: CP #,Total #
10.0,3433,9870,2496,15799.0
13.0,2557,7145,1710,11412.0
16.0,1746,5477,1349,8572.0
19.0,1386,4210,1122,6718.0
22.0,1141,3225,964,5330.0
25.0,956,2938,860,4754.0
Total,11219,32865,8501,52585.0
